In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

# Load Model

In [ ]:
MODEL_DIRECTORY = 'models'
MODEL = ''
VERSION = ''
tf_estimator = tf.saved_model.load(os.path.join(MODEL_DIRECTORY, MODEL, VERSION))

# Deploy Model

In [ ]:
tf_endpoint_name = 'ml-screen-reader-endpoint'
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                  instance_type='ml.m4.xlarge',
                                  endpoint_name=tf_endpoint_name)

In [ ]:
sess.delete_endpoint(endpoint_name=tf_endpoint_name)